In [2]:
PART = 50000

In [3]:
import pandas as pd
import numpy as np

In [4]:
train_data = pd.read_parquet("/kaggle/input/ecup-comp/train_data.parquet", engine='pyarrow')

In [5]:
train_data.sample(5)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,text_1,text_2,name_1,description_1,name_bert_1,name_2,description_2,name_bert_2
1095425,1265655085,1265655046,0,"[[0.1316687911748886, -0.4502332806587219, -0....","[[-0.057748183608055115, -0.28391164541244507,...",Одежда и обувь Обувь Повседневная обувь Туфли ...,Одежда и обувь Обувь Повседневная обувь Туфли ...,Туфли Tom&Miki Праздничные,"Туфли для девочки Tom&Miki, это стиль и комфор...","[-0.5111970901489258, 0.4222921133041382, 0.50...",Туфли Tom&Miki Праздничные,"Туфли для девочки Tom&Miki, это стиль и комфор...","[-0.511197030544281, 0.42229223251342773, 0.50..."
116222,1452490387,1452492568,0,"[[0.026426799595355988, 0.005911916494369507, ...","[[0.026426799595355988, 0.005911916494369507, ...","Книги и цифровые книги Книги Печатные книги, ж...","Книги и цифровые книги Книги Печатные книги, ж...",Математика. ВПР. 5-й класс. 10 тренировочных в...,Пособие предназначено для подготовки к всеросс...,"[-0.4883764684200287, 0.44133898615837097, 0.5...",Сборник задач по высшей математике с контрольн...,Сборник содержит свыше трех с половиной тысяч ...,"[-0.2987927496433258, 0.543397843837738, 0.426..."
982571,1498345847,1536933032,1,"[[-0.2053118795156479, 0.09080413728952408, -0...","[[-0.2053118795156479, 0.09080413728952408, -0...",Одежда и обувь Одежда Одежда Футболка Быт и до...,Одежда и обувь Одежда Одежда Футболка Быт и до...,Футболка Быт и досуг,Ткань/материал: хлопок/хлопковая<br>Содержание...,"[-0.37584683299064636, 0.5742789506912231, 0.6...",Футболка Быт и досуг,Ткань/материал: хлопок/хлопковая<br>Содержание...,"[-0.37584683299064636, 0.5742789506912231, 0.6..."
970547,1406354129,1355744294,0,"[[0.008713901042938232, 0.001336023211479187, ...","[[0.33102309703826904, 0.605811357498169, -0.1...",EPG Строительство и ремонт Светильники и аксес...,EPG Строительство и ремонт Светильники и аксес...,"TengWeiYun Прожектор, SL224-F 6500 К",(Теплый совет)\n\n* Время зарядки и время осве...,"[-0.46764886379241943, 0.6448335647583008, 0.5...",TengWeiYun Блок питания для светодиодной ленты...,* ON - По соображениям безопасности входные и ...,"[-0.49281570315361023, 0.5520446300506592, 0.5..."
1127498,1218102218,1297729089,0,"[[0.47922998666763306, -0.11029735952615738, -...","[[0.2799994647502899, -0.03963233530521393, -0...",EPG Красота и гигиена Косметика для ухода за в...,EPG Красота и гигиена Косметика для ухода за в...,Revlonissimo Colorsmetique Intense Blonde 1202...,Revlonissimo Colorsmetique Intense Blonde - эт...,"[-0.4274217188358307, 0.5643154382705688, 0.52...","Revlon Professional Краска для волос, 60 мл",Revlon Professional предлагает краску для воло...,"[-0.5963780879974365, 0.5443589091300964, 0.67..."


In [6]:
train_data.shape

(1168516, 13)

In [7]:
train_data.isna().sum()

variantid1               0
variantid2               0
target                   0
pic_embeddings_1         0
pic_embeddings_2         0
text_1                   0
text_2                   0
name_1                   0
description_1       136746
name_bert_1              0
name_2                   0
description_2       136241
name_bert_2              0
dtype: int64

In [8]:
train_data.fillna('unk', inplace=True)

In [9]:
import polars as pl
import torch
from transformers import AutoModel, AutoTokenizer
import re
from sklearn.decomposition import PCA
from tqdm import tqdm

In [54]:
df = train_data[:PART].copy()

In [55]:
df['name_descr_attr_1'] = df.apply(lambda x: x['name_1'] + ' ' + x['description_1'] + ' ' + x['text_1'] ,axis=1)
df['name_descr_attr_2'] = df.apply(lambda x: x['name_2'] + ' ' + x['description_2'] + ' ' + x['text_2'] ,axis=1)

In [56]:
df = df.drop(['name_1', 'description_1', 'text_1', 'name_2', 'description_2', 'text_2'], axis=1)
df.sample(3)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,name_bert_1,name_bert_2,name_descr_attr_1,name_descr_attr_2
47382,742017870,672215328,1,"[[-0.4017437696456909, -0.1539468765258789, 0....","[[-0.4100593626499176, -0.20682410895824432, 0...","[-0.521065354347229, 0.48297134041786194, 0.35...","[-0.3650519549846649, 0.3335617482662201, 0.42...",Газированный напиток Rich Bitter Гранат (РИЧ) ...,"Rich Bitter 0,33л.*36шт. Гранат, unk EPG Проду..."
20823,597414228,597413494,0,"[[-0.4684402346611023, 0.09774313867092133, 0....","[[-0.4269014000892639, 0.0950726717710495, 0.3...","[-0.6016467213630676, 0.4564497470855713, 0.40...","[-0.5988166928291321, 0.4389316439628601, 0.49...",Ambrella Light Комплект накладного светильник...,Ambrella Light Комплект накладного светильник...
34111,1259278201,1259281768,0,"[[0.049652427434921265, -0.37309375405311584, ...","[[0.01764094829559326, -0.4623051881790161, 1....","[-0.3984675705432892, 0.3324018120765686, 0.40...","[-0.3238489031791687, 0.41722023487091064, 0.5...",Шланг для Керхер штуцер-штуцер 10м. ПВХ нов. о...,Шланг для Керхер штуцер-штуцер 15м. ПВХ нов. о...


In [57]:
def remove_br_tags(text):
    text = re.sub(r'<br\s*/?>', '', text)
    text = re.sub(r'<li\s*/?>', '', text)
    text = re.sub(r'&laquo;|&raquo;|&minus;', '', text)
    return text

df = df.sort_values('variantid1')
df['name_descr_attr_1'] = df['name_descr_attr_1'].astype('str').apply(remove_br_tags)
df['name_descr_attr_1'] = df['name_descr_attr_1'].apply(lambda x: '' if x == "None" else x)
df['name_descr_attr_2'] = df['name_descr_attr_2'].astype('str').apply(remove_br_tags)
df['name_descr_attr_2'] = df['name_descr_attr_2'].apply(lambda x: '' if x == "None" else x)

In [58]:
model_id = "sergeyzh/rubert-tiny-turbo"
model = AutoModel.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [59]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Aug 22 13:40:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             36W /   70W |    1171MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [60]:
def tokenize_func(sample):
    return tokenizer(sample, padding=True, truncation=True, max_length=256, return_tensors='pt')

In [61]:
first = tokenize_func(df['name_descr_attr_1'][:PART].tolist())


In [62]:
second = tokenize_func(df['name_descr_attr_2'][:PART].tolist())

In [63]:
firts = {key: val.to('cuda') for key, val in first.items()}
second = {key: val.to('cuda') for key, val in second.items()}

In [64]:
f_emb, s_emb = [], []
batch_size = 32
all_embs = []
num_batches = first['input_ids'].size(0) // batch_size + (first['input_ids'].size(0) % batch_size != 0)

for i in tqdm(range(num_batches)):
    start = i * batch_size
    end = min((i + 1) * batch_size, first['input_ids'].size(0))
    

    input_ids_btch_f = first['input_ids'][start:end].to('cuda')
    token_type_ids_btch_f = first['token_type_ids'][start:end].to('cuda')
    attention_mask_btch_f = first['attention_mask'][start:end].to('cuda')
    
    input_ids_btch_s = second['input_ids'][start:end].to('cuda')
    token_type_ids_btch_s = second['token_type_ids'][start:end].to('cuda')
    attention_mask_btch_s = second['attention_mask'][start:end].to('cuda')
    
    with torch.no_grad():
        outputs_f = model(
            input_ids=input_ids_btch_f,
            token_type_ids=token_type_ids_btch_f,
            attention_mask=attention_mask_btch_f
        )
        outputs_s = model(
            input_ids=input_ids_btch_s,
            token_type_ids=token_type_ids_btch_s,
            attention_mask=attention_mask_btch_s
        )
    embs_f = outputs_f.pooler_output
    embs_s = outputs_s.pooler_output
    

    f_emb.append(embs_f.to('cuda'))
    s_emb.append(embs_s.to('cuda'))


f_emb = torch.cat(f_emb, dim=0).to('cuda')
s_emb = torch.cat(s_emb, dim=0).to('cuda')



100%|██████████| 1563/1563 [01:18<00:00, 19.83it/s]


In [65]:
df.sample(3)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,name_bert_1,name_bert_2,name_descr_attr_1,name_descr_attr_2
31755,726271976,726271513,0,"[[0.4348492920398712, -0.16435982286930084, 0....","[[0.6691069602966309, -0.3145756125450134, 0.4...","[-0.6367497444152832, 0.670048177242279, 0.296...","[-0.6616948246955872, 0.6568771600723267, 0.33...",Часы наручные мужские механические Восток Кома...,Часы наручные мужские механические Восток Кома...
10270,1431964236,1456480238,1,"[[1.3249273300170898, 0.3820229768753052, 0.11...","[[1.3249273300170898, 0.3820229768753052, 0.11...","[-0.6231591701507568, 0.43777358531951904, 0.5...","[-0.6231591701507568, 0.43777358531951904, 0.5...",Кроссовки unk Одежда и обувь Обувь Повседневна...,Кроссовки unk Одежда и обувь Обувь Повседневна...
34013,1452921114,1450512353,1,"[[-0.6794122457504272, -0.41956794261932373, 0...","[[-0.6794122457504272, -0.41956794261932373, 0...","[-0.6636412143707275, 0.5872519016265869, 0.55...","[-0.6636412143707275, 0.5872519016265869, 0.55...",Зимний камуфляжный костюм мужской на подкладке...,Зимний камуфляжный костюм мужской на подкладке...


In [66]:
first_emb = pd.Series([row for row in f_emb.to('cpu').numpy()])

In [67]:
second_emb = pd.Series([row for row in s_emb.to('cpu').numpy()])

In [68]:
df['new_embs_1'] = first_emb
df['new_embs_2'] = second_emb

In [69]:
df.sample(3)

,variantid1,variantid2,target,pic_embeddings_1,pic_embeddings_2,name_bert_1,name_bert_2,name_descr_attr_1,name_descr_attr_2,new_embs_1,new_embs_2
43737,1221179291,1185641734,1,"[[0.49464741349220276, -0.3034878075122833, -0...","[[0.4990813136100769, -0.3098682165145874, -0....","[-0.28899574279785156, 0.5935267210006714, 0.6...","[-0.3213390111923218, 0.5213029384613037, 0.75...",Бра Loft IT LOFT3110W В создании комфортного и...,"Loft it Бра, E27, 60 Вт Высота, мм - 400\n Диа...","[0.18729667, -0.22415912, 0.05588255, 0.080141...","[0.1902431, -0.21900587, 0.061495062, 0.073189..."
1389,132496408,132493364,0,"[[-0.5607186555862427, -0.08182933926582336, -...","[[-0.5607186555862427, -0.08182933926582336, -...","[-0.6563622355461121, 0.40832197666168213, 0.6...","[-0.6634868383407593, 0.393804669380188, 0.742...","Кисточка с краской для ремонта на ACURA, код Y...","Кисточка с краской для ремонта на ACURA, код Y...","[0.010780871, -0.16928965, 0.15414576, 0.09549...","[0.17028597, -0.25817138, 0.094255075, -0.0143..."
48665,292916647,59597556,0,"[[-0.9593510627746582, 0.09513594955205917, -0...","[[-1.212763786315918, -0.10293066501617432, -0...","[-0.47446146607398987, 0.5406014323234558, 0.5...","[-0.3763972222805023, 0.5342126488685608, 0.37...","Nivea Средство для душа, гель, 250 мл Кремовый...","Nivea Смываемый кондиционер для тела ""Медовое ...","[0.06882573, -0.12337289, 0.05140396, 0.068145...","[0.068902954, -0.11182501, 0.047731332, 0.0281..."


In [70]:
embs_pic_cols_1 = [f'pic_emb_1_{i}' for i in range(128)]
embs_pic_cols_2 = [f'pic_emb_2_{i}' for i in range(128)]

new_embs_cols_1 = [f'new_emb_1_{i}' for i in range(312)]
new_embs_cols_2 = [f'new_emb_2_{i}' for i in range(312)]

name_bert_cols_1 = [f'name_bert_1_{i}' for i in range(64)]
name_bert_cols_2 = [f'name_bert_2_{i}' for i in range(64)]

In [71]:
list_emb_pic_1= []
for i in range(PART):
    list_emb_pic_1.append(df['pic_embeddings_1'].iloc[i][0])
    
list_emb_pic_2= []
for i in range(PART):
    list_emb_pic_2.append(df['pic_embeddings_2'].iloc[i][0])

In [ ]:
pic_data_1 = pd.DataFrame(list_emb_pic_1, columns=embs_pic_cols_1)
pic_data_2 = pd.DataFrame(list_emb_pic_2, columns=embs_pic_cols_2)

new_emb_data_cols_1 = pd.DataFrame(df['new_embs_1'].tolist(), columns=new_embs_cols_1)
new_emb_data_cols_2 = pd.DataFrame(df['new_embs_2'].tolist(), columns=new_embs_cols_2)

name_bert_data_1 = pd.DataFrame(df['name_bert_1'].tolist(), columns=name_bert_cols_1)
name_bert_data_2 = pd.DataFrame(df['name_bert_2'].tolist(), columns=name_bert_cols_2)

In [79]:
dataset = pd.concat([df, name_bert_data_1, name_bert_data_2,pic_data_1,pic_data_2, new_emb_data_cols_1, new_emb_data_cols_2], axis=1).copy()


In [80]:
dataset = dataset.drop(['pic_embeddings_1', 'pic_embeddings_2','name_bert_1','name_bert_2','name_descr_attr_1','name_descr_attr_2','new_embs_1','new_embs_2'], axis=1)

In [81]:
dataset.sample(3)

,variantid1,variantid2,target,name_bert_1_0,name_bert_1_1,name_bert_1_2,name_bert_1_3,name_bert_1_4,name_bert_1_5,name_bert_1_6,...,new_emb_2_302,new_emb_2_303,new_emb_2_304,new_emb_2_305,new_emb_2_306,new_emb_2_307,new_emb_2_308,new_emb_2_309,new_emb_2_310,new_emb_2_311
45582,903342997,725193423,0,-0.391790,0.464231,0.447592,-0.483370,0.621167,0.585385,0.083697,...,-0.149986,-0.179225,-0.178673,-0.064123,-0.030811,0.001772,0.053568,0.030955,-0.115669,-0.069304
814,1559665795,1560570202,0,-0.451713,0.542138,0.494468,-0.328771,0.501080,0.754629,-0.034751,...,-0.044412,-0.101710,-0.117017,-0.142268,-0.011485,0.001106,0.111349,-0.104984,-0.025289,0.103674
7662,76745733,522916040,0,-0.564453,0.340865,0.060476,-0.505234,0.518960,0.667298,0.111722,...,-0.142517,-0.040717,-0.106600,-0.061686,-0.087730,0.001709,-0.001500,-0.034348,-0.126478,0.026082


In [83]:
dataset.shape

(50000, 1011)

In [84]:
dataset.to_csv('dataset.csv', index=False)